# Multivariate Time Series Forecasting: GARCH, LSTM, Transformer

This notebook demonstrates a systematic workflow for multivariate time series forecasting:
1. **Load Data**: Read `sample_multivar_timeseries.csv` with multiple features
2. **Install Dependencies**: Ensure all required packages are available
3. **Preprocessing**: Normalize, engineer features, and prepare sequences
4. **Models**: GARCH (univariate volatility), LSTM (RNN), Transformer (attention-based)
5. **Pipeline**: Create a unified training and evaluation pipeline
6. **Comparison**: Compare RMSE, MAE, and visualize predictions

**Target column**: `NIFTY` (close prices) — predicted using other columns as features.

In [ ]:
import sys
import subprocess

# Install required packages if not available
packages = {
    'arch': 'arch',
    'tensorflow': 'tensorflow',
    'torch': 'torch',
    'sklearn': 'scikit-learn',
    'numpy': 'numpy',
    'pandas': 'pandas',
    'matplotlib': 'matplotlib'
}

for import_name, pip_name in packages.items():
    try:
        __import__(import_name)
        print(f"✓ {pip_name} already installed")
    except ImportError:
        print(f"Installing {pip_name}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", pip_name])
        print(f"✓ {pip_name} installed")

print("\nAll dependencies ready!")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
sns.set_style("darkgrid")
plt.rcParams['figure.figsize'] = (14, 5)

%matplotlib inline

print("Imports successful!")

## 1. Load Data

Load the multivariate time series from CSV. The target column is `NIFTY` (close prices).

In [ ]:
# Load data
df = pd.read_csv('./data/sample_multivar_timeseries.csv', parse_dates=['date'] if 'date' in open('./data/sample_multivar_timeseries.csv').readline() else False)
print(f"Dataset shape: {df.shape}")
print(f"\nFirst few rows:\n{df.head()}")
print(f"\nData types:\n{df.dtypes}")
print(f"\nMissing values:\n{df.isnull().sum()}")
print(f"\nBasic statistics:\n{df.describe()}")

# Identify target column
TARGET_COLUMN = 'NIFTY'
if TARGET_COLUMN not in df.columns:
    print(f"\nWarning: '{TARGET_COLUMN}' not found. Available columns: {df.columns.tolist()}")
    # Try to find close price column
    close_cols = [col for col in df.columns if 'close' in col.lower() or 'nifty' in col.lower()]
    TARGET_COLUMN = close_cols[0] if close_cols else df.columns[-1]
    print(f"Using '{TARGET_COLUMN}' as target column")

print(f"\nTarget column: {TARGET_COLUMN}")

In [ ]:
# Visualize target column
fig, axes = plt.subplots(2, 1, figsize=(14, 8))

# Time series plot
axes[0].plot(df.index, df[TARGET_COLUMN], label=TARGET_COLUMN, linewidth=2)
axes[0].set_title(f'Time Series: {TARGET_COLUMN}', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Price')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Distribution
axes[1].hist(df[TARGET_COLUMN], bins=50, edgecolor='black', alpha=0.7)
axes[1].set_title(f'Distribution: {TARGET_COLUMN}', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Price')
axes[1].set_ylabel('Frequency')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 2. Data Preprocessing

Handle missing values, create training/test splits, normalize, and prepare sequences for deep learning models.

In [ ]:
# Handle missing values
df_clean = df.fillna(method='ffill').fillna(method='bfill')
print(f"Missing values after filling: {df_clean.isnull().sum().sum()}")

# Drop date column if present (for analysis)
if 'date' in df_clean.columns:
    df_clean = df_clean.drop('date', axis=1)

# Select numerical columns only
numerical_cols = df_clean.select_dtypes(include=[np.number]).columns.tolist()
df_clean = df_clean[numerical_cols]

print(f"Numerical features: {df_clean.shape[1]}")
print(f"Dataset shape: {df_clean.shape}")

# Normalize using MinMaxScaler (for LSTM/Transformer)
scaler_minmax = MinMaxScaler(feature_range=(0, 1))
df_scaled = pd.DataFrame(
    scaler_minmax.fit_transform(df_clean),
    columns=df_clean.columns
)

# Create train/test split (80/20)
TRAIN_SIZE = 0.8
split_idx = int(len(df_scaled) * TRAIN_SIZE)

df_train = df_scaled[:split_idx]
df_test = df_scaled[split_idx:]

print(f"\nTrain shape: {df_train.shape}")
print(f"Test shape: {df_test.shape}")
print(f"Train/Test split: {TRAIN_SIZE*100:.0f}% / {(1-TRAIN_SIZE)*100:.0f}%")

In [ ]:
# Create sequences for deep learning (sliding window approach)
def create_sequences(data, target_col_idx, window_size=30, forecast_horizon=1):
    """
    Create sequences for time series forecasting.
    
    Args:
        data: scaled DataFrame
        target_col_idx: index of target column in data
        window_size: number of timesteps to look back
        forecast_horizon: number of steps ahead to predict
    
    Returns:
        X, y: feature and target sequences
    """
    X, y = [], []
    for i in range(len(data) - window_size - forecast_horizon + 1):
        X.append(data.iloc[i:i+window_size].values)
        y.append(data.iloc[i+window_size+forecast_horizon-1, target_col_idx])
    return np.array(X), np.array(y)

# Parameters
WINDOW_SIZE = 30
FORECAST_HORIZON = 1
TARGET_COL_IDX = df_scaled.columns.get_loc(TARGET_COLUMN)

# Create sequences
X_train_seq, y_train_seq = create_sequences(
    df_train, TARGET_COL_IDX, WINDOW_SIZE, FORECAST_HORIZON
)
X_test_seq, y_test_seq = create_sequences(
    df_test, TARGET_COL_IDX, WINDOW_SIZE, FORECAST_HORIZON
)

print(f"X_train shape: {X_train_seq.shape} (samples, timesteps, features)")
print(f"y_train shape: {y_train_seq.shape}")
print(f"X_test shape: {X_test_seq.shape}")
print(f"y_test shape: {y_test_seq.shape}")

## 3. Model 1: GARCH (Univariate Volatility Model)

GARCH models volatility of returns. We fit it on training returns and produce short-term variance forecasts.

In [ ]:
from arch import arch_model

# Prepare GARCH: compute log returns from the original (unscaled) data
original_prices = df_clean[TARGET_COLUMN].values
returns = 100 * np.diff(np.log(original_prices))

# Fit GARCH(1,1) on training returns
train_returns = returns[:split_idx-1]
test_returns = returns[split_idx-1:]

print(f"Training returns shape: {train_returns.shape}")
print(f"Test returns shape: {test_returns.shape}")

# Fit GARCH model
garch_model = arch_model(train_returns, vol='Garch', p=1, q=1, dist='normal')
garch_result = garch_model.fit(disp='off')

print("\nGARCH Model Summary:")
print(garch_result.summary())

# Forecast conditional variance for test period
garch_forecast = garch_result.forecast(horizon=len(test_returns), reindex=False)
garch_conditional_var = garch_forecast.variance.values.flatten()

print(f"\nGARCH variance forecast shape: {garch_conditional_var.shape}")

In [ ]:
# Simple price forecast from GARCH: use mean return + variance scaling
# This is a simplified approach; in practice, combine with other models
garch_price_pred = df_clean[TARGET_COLUMN].iloc[split_idx:].values.copy()
for i in range(len(test_returns)):
    # Naive: use previous price + scaled variance
    mean_return = np.mean(train_returns)
    scaled_return = mean_return + (np.sqrt(garch_conditional_var[i]) * 0.01)
    garch_price_pred[i] = original_prices[split_idx + i - 1] * (1 + scaled_return / 100)

# Inverse transform to original scale for comparison
garch_price_pred_rescaled = garch_price_pred
y_test_original = scaler_minmax.inverse_transform(
    np.column_stack([df_test.iloc[WINDOW_SIZE:WINDOW_SIZE+len(y_test_seq), :].values])
)[:, 0]

print(f"GARCH predictions (original scale): {garch_price_pred_rescaled[:5]}")

## 4. Model 2: LSTM (Recurrent Neural Network)

LSTM captures temporal dependencies and patterns in multivariate sequences.

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Build LSTM model
def build_lstm_model(input_shape, lstm_units=64, dropout_rate=0.2):
    model = Sequential([
        LSTM(lstm_units, activation='relu', input_shape=input_shape, return_sequences=True),
        Dropout(dropout_rate),
        LSTM(lstm_units, activation='relu', return_sequences=False),
        Dropout(dropout_rate),
        Dense(32, activation='relu'),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

# Create and train LSTM
print("Building LSTM model...")
lstm_model = build_lstm_model((X_train_seq.shape[1], X_train_seq.shape[2]), lstm_units=64)
lstm_model.summary()

print("\nTraining LSTM...")
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lstm_history = lstm_model.fit(
    X_train_seq, y_train_seq,
    epochs=30,
    batch_size=32,
    validation_split=0.1,
    callbacks=[early_stop],
    verbose=1
)

print("LSTM training completed!")

In [ ]:
# Predict on test set
print("LSTM predictions on test set...")
lstm_pred_scaled = lstm_model.predict(X_test_seq, verbose=0)

# Inverse transform predictions to original scale
# Create dummy array with right shape for inverse transform
dummy = np.zeros((lstm_pred_scaled.shape[0], df_scaled.shape[1]))
dummy[:, TARGET_COL_IDX] = lstm_pred_scaled.flatten()
lstm_pred_original = scaler_minmax.inverse_transform(dummy)[:, TARGET_COL_IDX]

# Get test targets in original scale
dummy_test = np.zeros((y_test_seq.shape[0], df_scaled.shape[1]))
dummy_test[:, TARGET_COL_IDX] = y_test_seq
y_test_original = scaler_minmax.inverse_transform(dummy_test)[:, TARGET_COL_IDX]

print(f"LSTM predictions shape: {lstm_pred_original.shape}")
print(f"Test targets shape: {y_test_original.shape}")
print(f"First 5 predictions: {lstm_pred_original[:5]}")
print(f"First 5 targets: {y_test_original[:5]}")

# Plot training history
fig, axes = plt.subplots(1, 2, figsize=(14, 4))
axes[0].plot(lstm_history.history['loss'], label='Training Loss')
axes[0].plot(lstm_history.history['val_loss'], label='Validation Loss')
axes[0].set_title('LSTM Training History - Loss')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].plot(lstm_history.history['mae'], label='Training MAE')
axes[1].plot(lstm_history.history['val_mae'], label='Validation MAE')
axes[1].set_title('LSTM Training History - MAE')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('MAE')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. Model 3: Transformer (Attention-Based)

A simple Transformer-style model using MultiHeadAttention for capturing global patterns in sequences.

In [ ]:
from tensorflow.keras.layers import MultiHeadAttention, LayerNormalization, GlobalAveragePooling1D

def build_transformer_model(input_shape, head_size=32, num_heads=2, ff_dim=64, dropout_rate=0.1):
    """
    Build a simple Transformer-style model for time series forecasting.
    """
    inputs = tf.keras.Input(shape=input_shape)
    x = inputs
    
    # Multi-head self-attention
    attention_output = MultiHeadAttention(
        num_heads=num_heads, 
        key_dim=head_size,
        dropout=dropout_rate
    )(x, x)
    x = LayerNormalization(epsilon=1e-6)(x + attention_output)
    
    # Feed-forward network
    ff_output = Dense(ff_dim, activation='relu')(x)
    ff_output = Dropout(dropout_rate)(ff_output)
    ff_output = Dense(input_shape[-1])(ff_output)
    x = LayerNormalization(epsilon=1e-6)(x + ff_output)
    
    # Global pooling and output
    x = GlobalAveragePooling1D()(x)
    x = Dense(64, activation='relu')(x)
    x = Dropout(dropout_rate)(x)
    outputs = Dense(1)(x)
    
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

# Create and train Transformer
print("Building Transformer model...")
transformer_model = build_transformer_model(
    (X_train_seq.shape[1], X_train_seq.shape[2]),
    head_size=16, num_heads=4, ff_dim=64
)
transformer_model.summary()

print("\nTraining Transformer...")
transformer_history = transformer_model.fit(
    X_train_seq, y_train_seq,
    epochs=30,
    batch_size=32,
    validation_split=0.1,
    callbacks=[early_stop],
    verbose=1
)

print("Transformer training completed!")

In [ ]:
# Predict on test set
print("Transformer predictions on test set...")
transformer_pred_scaled = transformer_model.predict(X_test_seq, verbose=0)

# Inverse transform predictions to original scale
dummy_trans = np.zeros((transformer_pred_scaled.shape[0], df_scaled.shape[1]))
dummy_trans[:, TARGET_COL_IDX] = transformer_pred_scaled.flatten()
transformer_pred_original = scaler_minmax.inverse_transform(dummy_trans)[:, TARGET_COL_IDX]

print(f"Transformer predictions shape: {transformer_pred_original.shape}")
print(f"First 5 predictions: {transformer_pred_original[:5]}")

# Plot training history
fig, axes = plt.subplots(1, 2, figsize=(14, 4))
axes[0].plot(transformer_history.history['loss'], label='Training Loss')
axes[0].plot(transformer_history.history['val_loss'], label='Validation Loss')
axes[0].set_title('Transformer Training History - Loss')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].plot(transformer_history.history['mae'], label='Training MAE')
axes[1].plot(transformer_history.history['val_mae'], label='Validation MAE')
axes[1].set_title('Transformer Training History - MAE')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('MAE')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Model Comparison

Compare RMSE, MAE, and R² scores across all three models.

In [ ]:
def compute_metrics(y_true, y_pred, model_name):
    """Compute standard regression metrics."""
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100  # Mean Absolute Percentage Error
    
    print(f"\n{model_name}")
    print(f"  RMSE: {rmse:.4f}")
    print(f"  MAE:  {mae:.4f}")
    print(f"  MAPE: {mape:.2f}%")
    print(f"  R²:   {r2:.4f}")
    
    return {'Model': model_name, 'RMSE': rmse, 'MAE': mae, 'MAPE': mape, 'R2': r2}

# Compute metrics for all models
results = []

# GARCH (adjust length if needed)
garch_pred_aligned = garch_price_pred_rescaled[:len(y_test_original)]
results.append(compute_metrics(y_test_original, garch_pred_aligned, "GARCH"))

# LSTM
results.append(compute_metrics(y_test_original, lstm_pred_original, "LSTM"))

# Transformer
results.append(compute_metrics(y_test_original, transformer_pred_original, "Transformer"))

# Create comparison DataFrame
comparison_df = pd.DataFrame(results)
print("\n" + "="*70)
print("MODEL COMPARISON")
print("="*70)
print(comparison_df.to_string(index=False))
print("="*70)

In [ ]:
# Visualize metric comparison
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# RMSE comparison
ax = axes[0, 0]
ax.bar(comparison_df['Model'], comparison_df['RMSE'], color=['#1f77b4', '#ff7f0e', '#2ca02c'])
ax.set_title('RMSE Comparison (Lower is Better)', fontsize=12, fontweight='bold')
ax.set_ylabel('RMSE')
ax.grid(True, alpha=0.3, axis='y')
for i, v in enumerate(comparison_df['RMSE']):
    ax.text(i, v + 0.1, f'{v:.4f}', ha='center', va='bottom')

# MAE comparison
ax = axes[0, 1]
ax.bar(comparison_df['Model'], comparison_df['MAE'], color=['#1f77b4', '#ff7f0e', '#2ca02c'])
ax.set_title('MAE Comparison (Lower is Better)', fontsize=12, fontweight='bold')
ax.set_ylabel('MAE')
ax.grid(True, alpha=0.3, axis='y')
for i, v in enumerate(comparison_df['MAE']):
    ax.text(i, v + 0.02, f'{v:.4f}', ha='center', va='bottom')

# MAPE comparison
ax = axes[1, 0]
ax.bar(comparison_df['Model'], comparison_df['MAPE'], color=['#1f77b4', '#ff7f0e', '#2ca02c'])
ax.set_title('MAPE Comparison (Lower is Better)', fontsize=12, fontweight='bold')
ax.set_ylabel('MAPE (%)')
ax.grid(True, alpha=0.3, axis='y')
for i, v in enumerate(comparison_df['MAPE']):
    ax.text(i, v + 0.5, f'{v:.2f}%', ha='center', va='bottom')

# R² comparison
ax = axes[1, 1]
ax.bar(comparison_df['Model'], comparison_df['R2'], color=['#1f77b4', '#ff7f0e', '#2ca02c'])
ax.set_title('R² Score Comparison (Higher is Better)', fontsize=12, fontweight='bold')
ax.set_ylabel('R² Score')
ax.set_ylim([min(comparison_df['R2']) - 0.1, 1.0])
ax.grid(True, alpha=0.3, axis='y')
for i, v in enumerate(comparison_df['R2']):
    ax.text(i, v + 0.02, f'{v:.4f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

In [ ]:
# Time series prediction visualization
fig, ax = plt.subplots(figsize=(16, 6))

# Plot actual values
ax.plot(range(len(y_test_original)), y_test_original, 
        'o-', label='Actual', linewidth=2, markersize=4, color='black', alpha=0.7)

# Plot model predictions
ax.plot(range(len(lstm_pred_original)), lstm_pred_original, 
        's--', label='LSTM', linewidth=1.5, markersize=3, alpha=0.7)
ax.plot(range(len(transformer_pred_original)), transformer_pred_original, 
        '^--', label='Transformer', linewidth=1.5, markersize=3, alpha=0.7)
ax.plot(range(len(garch_pred_aligned)), garch_pred_aligned, 
        '*--', label='GARCH', linewidth=1.5, markersize=6, alpha=0.7)

ax.set_title(f'{TARGET_COLUMN}: Predictions vs Actual (Test Set)', fontsize=14, fontweight='bold')
ax.set_xlabel('Time Step')
ax.set_ylabel('Price')
ax.legend(loc='best', fontsize=11)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 7. Residual Analysis

Analyze prediction errors to understand model strengths and weaknesses.

In [ ]:
# Compute residuals
residuals_lstm = y_test_original - lstm_pred_original
residuals_transformer = y_test_original - transformer_pred_original
residuals_garch = y_test_original - garch_pred_aligned

# Plot residuals
fig, axes = plt.subplots(3, 2, figsize=(16, 10))

# LSTM residuals
axes[0, 0].plot(residuals_lstm, 'o-', alpha=0.7, color='#ff7f0e')
axes[0, 0].axhline(y=0, color='k', linestyle='--', linewidth=1)
axes[0, 0].set_title('LSTM: Residuals over time', fontweight='bold')
axes[0, 0].set_ylabel('Residual')
axes[0, 0].grid(True, alpha=0.3)

axes[0, 1].hist(residuals_lstm, bins=30, edgecolor='black', alpha=0.7, color='#ff7f0e')
axes[0, 1].set_title('LSTM: Residuals Distribution', fontweight='bold')
axes[0, 1].set_xlabel('Residual')
axes[0, 1].set_ylabel('Frequency')
axes[0, 1].grid(True, alpha=0.3, axis='y')

# Transformer residuals
axes[1, 0].plot(residuals_transformer, 'o-', alpha=0.7, color='#2ca02c')
axes[1, 0].axhline(y=0, color='k', linestyle='--', linewidth=1)
axes[1, 0].set_title('Transformer: Residuals over time', fontweight='bold')
axes[1, 0].set_ylabel('Residual')
axes[1, 0].grid(True, alpha=0.3)

axes[1, 1].hist(residuals_transformer, bins=30, edgecolor='black', alpha=0.7, color='#2ca02c')
axes[1, 1].set_title('Transformer: Residuals Distribution', fontweight='bold')
axes[1, 1].set_xlabel('Residual')
axes[1, 1].set_ylabel('Frequency')
axes[1, 1].grid(True, alpha=0.3, axis='y')

# GARCH residuals
axes[2, 0].plot(residuals_garch, 'o-', alpha=0.7, color='#1f77b4')
axes[2, 0].axhline(y=0, color='k', linestyle='--', linewidth=1)
axes[2, 0].set_title('GARCH: Residuals over time', fontweight='bold')
axes[2, 0].set_ylabel('Residual')
axes[2, 0].set_xlabel('Time Step')
axes[2, 0].grid(True, alpha=0.3)

axes[2, 1].hist(residuals_garch, bins=30, edgecolor='black', alpha=0.7, color='#1f77b4')
axes[2, 1].set_title('GARCH: Residuals Distribution', fontweight='bold')
axes[2, 1].set_xlabel('Residual')
axes[2, 1].set_ylabel('Frequency')
axes[2, 1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

# Print residual statistics
print("Residual Statistics")
print("="*70)
print(f"LSTM - Mean: {np.mean(residuals_lstm):.6f}, Std: {np.std(residuals_lstm):.6f}")
print(f"Transformer - Mean: {np.mean(residuals_transformer):.6f}, Std: {np.std(residuals_transformer):.6f}")
print(f"GARCH - Mean: {np.mean(residuals_garch):.6f}, Std: {np.std(residuals_garch):.6f}")
print("="*70)

## 8. Summary and Recommendations

**Key Findings:**
- LSTM and Transformer models leverage multivariate features for better predictions.
- GARCH models volatility rather than price levels; it complements deep learning models.
- The best model depends on data characteristics, target use case, and available computational resources.

**Next Steps (for production):**
1. **Hyperparameter tuning**: Use GridSearch or Optuna for optimal parameters.
2. **Ensemble methods**: Combine predictions from multiple models.
3. **Longer horizons**: Test multi-step-ahead forecasting.
4. **Feature engineering**: Add technical indicators (RSI, MACD, Bollinger Bands, etc.).
5. **External features**: Incorporate macroeconomic or sector-specific data.
6. **Cross-validation**: Use time-series-aware CV (e.g., walk-forward validation).
7. **Model deployment**: Serve models via FastAPI or Docker.

**References:**
- [Transformer for Time Series](https://keras.io/examples/timeseries/timeseries_transformer_classification/)
- [GARCH Models](https://arch.readthedocs.io/)
- [LSTM for Time Series](https://keras.io/examples/timeseries/lstm_seq2seq/)